In [1]:
# Escenario 2 - Balancear la cantidad de registros según la variable target
# selecciono 150.000 registros por cada opción de mi target, 
# entonces son 150.000 con SI y 50.000 con NO (300.000 registros del total)

In [2]:
# Cargar funciones de la librería de python data analysis
import pandas as pd 

# Leer csv con datos y cargar en el dataframe data
# data = pd.read_csv("data/analisis_desercion_2018.csv", sep=';', encoding = "ISO-8859-1") 
data2018 = pd.read_csv("data/analisis_desercion_2018.csv", sep=';', encoding = "ISO-8859-1") 
data2019 = pd.read_csv("data/analisis_desercion_2019.csv", sep=';', encoding = "ISO-8859-1") 
data2020 = pd.read_csv("data/analisis_desercion_2020.csv", sep=';', encoding = "ISO-8859-1") 

In [3]:
# concatenar los 3 datasets
data = pd.concat([data2018,data2019,data2020])
# Preview de las 5 primeras filas de data 
data.head()

,estudiante_id,grado_curso,edad,tiene_sobreedad,tipo_documento,sexo,es_indigena,idioma,tipo_gestion_id,tipo_zona_id,...,cuantas_personas_vive,cuantas_piezas_dormir,recibe_beneficio,como_se_traslada_escuela,cuanto_tiempo_tarda,recibe_alimentacion,nivel_pobreza,matriculado_periodo_actual,estudiante_id_posible,desertor
0,3,11,13,0,1,1,0,Castellano,1,1,...,4,1,1,Caminando,Menos de 30 minutos,0,3.420555,1,NaN,0
1,4,6,8,0,1,2,0,Guaraní,1,1,...,4,1,0,Caminando,Menos de 30 minutos,0,20.333610,1,NaN,0
2,5,7,8,0,1,1,0,Castellano,1,1,...,4,2,0,Caminando,Menos de 30 minutos,0,23.435420,1,NaN,0
3,78,6,7,0,1,1,0,Castellano,1,1,...,4,2,0,En moto,Menos de 30 minutos,0,11.506870,0,NaN,1
4,13,7,8,0,1,1,0,Castellano,3,1,...,4,4,0,Caminando,Menos de 30 minutos,0,14.552570,1,NaN,0


In [4]:
# Identificar variables categoricas
df = data
df.dtypes[df.dtypes=='object']

idioma                      object
como_se_traslada_escuela    object
cuanto_tiempo_tarda         object
dtype: object

In [5]:
# cargar codificador por etiquetas (LabelEncoder)
from sklearn import preprocessing

# Codificar cada variable categorica con su propio encoder --> utilizo .astype(str) para convertir los valores a string
leIdioma = preprocessing.LabelEncoder()
df["idioma"] = leIdioma.fit_transform(df["idioma"].astype(str))
leComoSeTrasladaEscuela = preprocessing.LabelEncoder()
df["como_se_traslada_escuela"] = leComoSeTrasladaEscuela.fit_transform(df["como_se_traslada_escuela"].astype(str))
leCuantoTiempoTarda = preprocessing.LabelEncoder()
df["cuanto_tiempo_tarda"] = leCuantoTiempoTarda.fit_transform(df["cuanto_tiempo_tarda"].astype(str))

df.head()

,estudiante_id,grado_curso,edad,tiene_sobreedad,tipo_documento,sexo,es_indigena,idioma,tipo_gestion_id,tipo_zona_id,...,cuantas_personas_vive,cuantas_piezas_dormir,recibe_beneficio,como_se_traslada_escuela,cuanto_tiempo_tarda,recibe_alimentacion,nivel_pobreza,matriculado_periodo_actual,estudiante_id_posible,desertor
0,3,11,13,0,1,1,0,0,1,1,...,4,1,1,0,1,0,3.420555,1,NaN,0
1,4,6,8,0,1,2,0,1,1,1,...,4,1,0,0,1,0,20.333610,1,NaN,0
2,5,7,8,0,1,1,0,0,1,1,...,4,2,0,0,1,0,23.435420,1,NaN,0
3,78,6,7,0,1,1,0,0,1,1,...,4,2,0,2,1,0,11.506870,0,NaN,1
4,13,7,8,0,1,1,0,0,3,1,...,4,4,0,0,1,0,14.552570,1,NaN,0


In [6]:
# Carga decision tree, holdout split y metricas
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

###########################################################
### opcion balanceando de acuerdo al valor de desertor (misma cantidad de registros)
df1 = df[df['desertor'] == 1] 
df0 = df[df['desertor'] == 0] 

df1 = df1.iloc[:150000]
df0 = df0.iloc[:150000]

# concatenar los 2 df
dflimit = pd.concat([df0,df1])

# Split en train y test
X = dflimit.iloc[:,1:28]
y = dflimit['desertor']
###########################################################


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Entrenar decision tree con entropia, size minimo de nodo 200 y profundidad maxima 12
clf = DecisionTreeClassifier(criterion="entropy", min_samples_leaf=200, max_depth=12)
clf = clf.fit(X_train,y_train)

# Predecir con datos de test
y_pred = clf.predict(X_test)

# Accuracy: (tp+tn)/n
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8533111111111111


In [7]:
# cargar exportador de grafos y funcion de llamada a sistema
from sklearn.tree import export_graphviz
# Exportar arbol
export_graphviz(clf, out_file="data/desercion.dot",  
                filled=True, rounded=True,
                special_characters=True, feature_names = X.columns,class_names = ['SI','NO'])


In [8]:
import pydot
(graph,) = pydot.graph_from_dot_file('data/desercion.dot')
graph.write_png('data/desercion_v2 - todos.png')

In [9]:
# Calcular matriz de confusion
#metrics.confusion_matrix(y_test, y_pred)
pd.crosstab(y_test, y_pred, 
            rownames=['actual'], 
            colnames=['pred'], margins=False, margins_name="Total")

pred,0,1
actual,,
0,39415,5787
1,7415,37383


In [10]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred)) 

              precision    recall  f1-score   support

           0       0.84      0.87      0.86     45202
           1       0.87      0.83      0.85     44798

    accuracy                           0.85     90000
   macro avg       0.85      0.85      0.85     90000
weighted avg       0.85      0.85      0.85     90000



In [11]:
# Obtener importancia de variables y vertificar variables mas relevantes
#clf.feature_importances_
fi = pd.DataFrame(zip(X.columns,clf.feature_importances_), columns=['feature','importance'])

fi[fi['importance'] > 0.0].sort_values(by=['importance'], ascending=False)

,feature,importance
1,edad,0.347708
3,tipo_documento,0.304066
12,aprobado_complementario,0.125959
0,grado_curso,0.049645
20,cuantas_personas_vive,0.041437
25,recibe_alimentacion,0.040058
21,cuantas_piezas_dormir,0.023472
11,aprobado_ordinario,0.015703
9,departamento_id,0.012904
2,tiene_sobreedad,0.006767
